In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob


pd.options.display.max_rows = 400
pd.options.display.max_colwidth = 4000

In [ ]:
jsons = []

for filename in glob('/Users/remim/tmp/cdtn-2020/*.json'):
    print(filename)
    jsons.append(pd.read_json(filename))

#actions = pd.read_json("/Users/remim/dev/cdtn/cdtn-monolog/weeks_1-4_2020.json")
#actions = pd.read_json("/Users/remim/tmp/catherine-logs.json")
#actions = pd.read_json("/Users/remim/tmp/all-2020.json")

actions = pd.concat(jsons, ignore_index=True)


In [ ]:
actions = pd.read_json('/Users/remim/tmp/cdtn-2020/2020-01-28.json')

In [ ]:
# reset idVisit in order to ensure unicity
actions['idVisit'] = actions['idVisit'].astype(str) + actions["lastActionDateTime"].astype(str)

In [ ]:
#actions = pd.read_json("/Users/remim/dev/cdtn/cdtn-monolog/2weeks.json")


In [ ]:
#actions = pd.read_json("/Users/remim/tmp/logs-1912.json")
#actions = pd.read_json("/Users/remim/data/cdtn-logs/daily/2019-12-19-log.json")

# Check visits and action per date : assert no issues during collection

In [ ]:
actions_dates = actions.drop_duplicates(subset=['idVisit'])['timestamp'].apply(lambda x : x.strftime("%d/%m/%y"))

In [ ]:
# we retrieve the sorted dates
import datetime
dates = [datetime.datetime.strptime(ts, "%d/%m/%y") for ts in actions_dates.unique()]
dates.sort()
sorteddates = [datetime.datetime.strftime(ts, "%d/%m/%y") for ts in dates]

In [ ]:
plot = sns.catplot(x="timestamp", kind="count", palette="ch:.25", aspect=25.7/10.27, data=pd.DataFrame(actions_dates), order=sorteddates);
plt.subplots_adjust(top=0.9)
plot.fig.suptitle("Visites par jour")
#plot.set_xticklabels(plot.get_xticklabels(), rotation=45)
#[sns.setp(ax.get_xticklabels(), rotation=45) for ax in plot.axes.flat]
for ax in plot.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

In [ ]:
plot = sns.catplot(x="timestamp", kind="count", palette="ch:.25", aspect=25.7/10.27, data=pd.DataFrame(actions_dates), order=sorteddates);
plt.subplots_adjust(top=0.9)
plot.fig.suptitle("Visites par jour")
#plot.set_xticklabels(plot.get_xticklabels(), rotation=45)
#[sns.setp(ax.get_xticklabels(), rotation=45) for ax in plot.axes.flat]
for ax in plot.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

# Most popular contents

In [ ]:
actions[actions.type == 'visit_content'][actions.url.apply(lambda x : x.startswith("https://code.travail.gouv.fr/"))]["url"]\
.apply(lambda x : x.split("gouv.fr/")[1])\
.value_counts()[:70]


# Datafiller : generer variantes pour ajout aux suggestions

In [ ]:
# import json
# with open('request.datafiller-1712.json') as fd:
#      df_reqs_json = json.load(fd)

In [ ]:
# titles_col = []
# variant_col = []
# is_title = []
# to_keep = []

# for entry in df_reqs_json['data']:
#     title = entry['title']
    
#     # first we add the title
#     titles_col.append(title)
#     variant_col.append(title)
#     is_title.append(True)

#     # then we add the variants
#     variants = entry['variants'].split('\n') if 'variants' in entry else []
#     for variant in variants :
#         titles_col.append(title)
#         variant_col.append(variant)
#         is_title.append(False)

# to_keep = np.ones([len(titles_col)], dtype=bool)        
# weight = [5] * len(titles_col)
         
# df_reqs_df = pd.DataFrame({'Candidat':variant_col, 'Entrée':titles_col, 'Titre':is_title, 'Conserver':to_keep, 'Poids': weight})

In [ ]:
#df_reqs_df.to_excel("df_suggestion_qualification.xlsx")
#df_reqs_df.to_csv("df_suggestion_qualification-1712.csv")

# Rank and scoring

In [ ]:
result_candidates_actions = actions[actions.type == "result_candidates"]
result_candidates_actions['documents'] = result_candidates_actions.result_candidates.apply(lambda x : x['documents'])

In [ ]:
unique_search_visit = result_candidates_actions.idVisit.unique()

In [ ]:
unique_search_visit.shape

In [ ]:
#one_visit = unique_search_visit[0]
#actions[actions.idVisit == 125978]

# here we could compare select_result and visit content in order to 
# add time spent (and categorize visits : short / medium / long ...)
actions[actions.type == "select_result"][:10]
actions[actions.type == "visit_content"][:10]

In [ ]:
# for each visit containing candidates actions
# we want to associate candidates and select result

# check if selection in previous candidates
def in_candidates(select_action, candidates):
    selected_url = select_action.res_selection['url']
    candidate_urls = [c['slug'] for c in candidates['documents']]
    
    if selected_url in candidate_urls:
        return (True, candidate_urls.index(selected_url))
    else :
        # elif : here we can be in different cases :
        # query undefined : theme results
        # selection resultat code du travail ou theme
        return (False, -1)

def analyze_visit(id_visit) :
    visit = actions[actions.idVisit == id_visit]

    last_candidates = {}
    last_query = ''
    consumed = False
    
    logs = []

    for _, action in visit.iterrows():
        if action.type == 'result_candidates':
            if last_candidates != action.result_candidates and last_query != action.query:
                last_candidates = action.result_candidates
                last_query = action.query
                consumed = False

        if (action.type == 'select_result'):
            
            # case no current candidates and selection
            if not(last_candidates):
                print('Selection before candidates ' + str(id_visit))
                continue;
            
#             selection = action.res_selection['url']
#             split = selection.split("/");
#             transformed = split[1].replace('-', "_").replace("fiche", "fiches") + "/" + split[2];                        
    
            (found, index) = in_candidates(action, last_candidates)
    
            if not found:
                print('Selection not in candidates ' + str(id_visit))
                #print(action.res_selection)
                #print([c['slug'] for c in last_candidates['documents']])
                
            # we keep track of the fact that candidates were "consumed"
            if index >= 0:  
                consumed = True
                
            logs.append({
                'docs_res': last_candidates['documents'],
                'articles_res': last_candidates['articles'],
                'themes_res': last_candidates['themes'],
                'idVisit': id_visit,
                'rank': index,
                'selection': action.res_selection['url'],
                'query': last_query
            })
                                

    if not consumed :
        print("Not consumed " + str(id_visit))
        logs.append({
                    'docs_res': last_candidates['documents'],
                    'articles_res': last_candidates['articles'],
                    'themes_res': last_candidates['themes'],
                    'idVisit': id_visit,
                    'rank': -2,
                    'selection': np.NaN,
                    'query': last_query
                })
        
    return logs


In [ ]:
logs = []
for i in unique_search_visit:
    print(i)
    logs += analyze_visit(i)

In [ ]:
df_searches = pd.DataFrame(logs)

In [ ]:
df_searches.to_excel("searches_candidates_selection_rank.xlsx")

In [ ]:
df_searches[df_searches['rank'] >= 0].shape[0] / df_searches.shape[0]

In [ ]:
def print_rangs(query):
    dem_queries = df_searches[(df_searches['query']==query) | (df_searches['query']==query)]
    only_pos = dem_queries[dem_queries['rank'] >=0]
    plot = sns.catplot(x="rank", kind="count", palette="ch:.25", data=pd.DataFrame(only_pos));
    plt.subplots_adjust(top=0.9)
    plot.fig.suptitle("Rangs pour la requête : " + query)

In [ ]:
print_rangs('congés sans solde')

In [ ]:
pd.Series(list(filter(lambda x : x >=0, pd.DataFrame(logs)['rank'].values.tolist()))).describe()
sns.catplot(x="rank", kind="count", palette="ch:.25", data=pd.DataFrame(logs));

In [ ]:
actions[actions.idVisit == 67888]

In [ ]:
counts = df_searches['query'].apply(lambda x : str.lower(x)).value_counts()[1:].rename_axis('unique_values').reset_index(name='counts')
counts[counts.counts > 1].to_excel("most_popular_queries.xlsx")

In [ ]:
#in_search
actions[actions.idVisit == 66730]

In [ ]:
plot = sns.catplot(y="referrerTypeName", kind="count", palette="ch:.25", data=actions)#.set_title('Provenance des visites');
plt.subplots_adjust(top=0.9)
plot.fig.suptitle('Provenance des visites')



In [ ]:
plot = sns.catplot(y="type", kind="count", palette="ch:.25", data=actions, aspect=25.7/10.27, order=['visit', 'home', 'result_candidates', 'search', 'select_result', 'suggestion_candidates', 'select_suggestion', 'outlink', 'download', 'external_search', 'feedback']);
plt.subplots_adjust(top=0.9)
plot.fig.suptitle("Types d'actions")


In [ ]:
actions[actions.type == 'search'].shape[0]

In [ ]:
visitLength = actions.groupby(["idVisit", "referrerTypeName"]).size().reset_index(name='countActions')

In [ ]:
visitLength[visitLength.countActions > 3]

In [ ]:
actions[actions.type == "select_suggestion"].prefix.apply(lambda x: len(x)).describe()


In [ ]:
actions.type.value_counts()

In [ ]:
result_candidates = actions[actions.type == "result_candidates"]
result_candidates[result_candidates["query"] != "undefined"].shape
#result_candidates["query"]

In [ ]:
actions.shape


In [ ]:
actions[actions.lastActionTimestamp == actions.lastActionTimestamp.max()]['timestamp']


In [ ]:
actions[actions.lastActionTimestamp == actions.lastActionTimestamp.min()]['timestamp']

In [ ]:
vl = visitLength[visitLength.countActions<30]

In [ ]:
#f, ax = plt.subplots(figsize=(20, 3))
#sns.set(rc={'figure.figsize':(11.1,80.0)})
plot = sns.catplot(x="countActions", kind="count", data=vl, aspect=20.7/8.27);
plt.subplots_adjust(top=0.9)
plot.fig.suptitle("Nombre d'actions")

In [ ]:
visitLength[visitLength.countActions == 1].referrerTypeName.value_counts()


In [ ]:
visitLength[visitLength.countActions == 2].referrerTypeName.value_counts()



In [ ]:
cdt = actions[actions.type == "select_result"]
#cdt[cdt['query'] == "licenciement"].idVisit
cdt.shape


In [ ]:
visitLength[visitLength.countActions == 7]



In [ ]:
set(actions[actions.type=='select_result'][:20].idVisit.tolist())


In [ ]:
actions[actions.type=='select_suggestion'].sugg_selection.value_counts()[:20]



In [ ]:
actions.type.unique()


In [ ]:
# get id visits where contains at least a select search result or a select suggestion
actions[(actions.type == "select_result") | (actions.type == "select_suggestion")].idVisit.unique()

In [ ]:
actions.idVisit.unique().shape

In [ ]:
id_results = actions[actions.type == "result_candidates"].idVisit.unique()

In [ ]:
id_select = actions[actions.type == "select_result"].idVisit.unique()

In [ ]:
id_results[:10]

In [ ]:
id_select[:10]

In [ ]:
actions[actions.type == "select_result"].res_selection[:1]

In [ ]:
id_results.shape

## Suggestions

In [ ]:
# number of suggestions selection vs number of searches

In [ ]:
n_unique_suggestions = actions[(actions.type == "select_suggestion")][["sugg_selection", "idVisit", "type"]].drop_duplicates()


In [ ]:
# isolate unique searches (to avoid 'return to search')
n_unique_searches = actions[(actions.type == "search")][["query", "idVisit", "type"]].drop_duplicates()
# remove random to mark the positive suggestions and only get right number or non suggested
non_suggested = n_unique_searches[n_unique_suggestions.shape[0]:]
# TODO isolate actual searches without suggestion (not only the count)

In [ ]:
sugg_data = pd.concat([n_unique_suggestions, non_suggested])
sugg_data['Search type'] = sugg_data.type.apply(lambda x : 'Recherche suggérée' if x != 'search' else 'Recherche directe')
plot = sns.catplot(y="Search type", kind="count", palette="ch:.25", data=sugg_data);
plt.subplots_adjust(top=0.9)
plot.fig.suptitle("Nombre de recherches directes & recherches suggérées")

In [ ]:
# prefix size

In [ ]:
prefix_size = pd.DataFrame(actions[actions.type=='select_suggestion'].prefix.apply(lambda x : len(x)))
prefix_size = prefix_size[prefix_size<50]
sns.catplot(x="prefix", kind="violin", data=prefix_size);
plt.subplots_adjust(top=0.8)
plot.fig.suptitle("Longueur du prefix de suggestion.")

## Results selection : check number of : pre-qualified / semantic / fulltext

In [ ]:
def get_type(element, keyword='url'):
    #parsed = json.loads(element)
    if 'algo' in element:
        return element['algo']
    elif element[keyword].startswith('http'):
        return 'http'
    else:
        # special case for theme search
        t = element[keyword][1:element[keyword].rfind('/')]
        if t == 'fiche-service-public' or t == 'fiche-ministere-travail' :
            return 'theme'
        else:
            return t
        
select_actions = actions[(actions.type == "select_result") & (actions.query != "undefined")]
selection_types = select_actions.res_selection.apply(lambda e : get_type(e))

In [ ]:
st_df = pd.DataFrame({'selection_type':selection_types})
st_df["search_engine"] = st_df['selection_type'].apply(lambda x : 'search engine' if x in ['fulltext', 'both', 'semantic'] else x)

plot = sns.catplot(y='selection_type', kind="count", data=st_df, order=['both', 'fulltext', 'semantic']);
                    #order=['pre-qualified', 'search engine'])

plt.subplots_adjust(top=0.9)
#plot.fig.suptitle("Répartition des selections pre-qualified / moteur de recherche.")
plot.fig.suptitle("Répartition des sélections des résultats du moteur de recherche.")

## Number of results in candidates for each type

In [ ]:
def read_candidates(cands):
    docs = cands['documents']
    themes = cands['themes']
    
    doc_types = [get_type(d, 'slug') for d in docs]
    theme_types = [get_type(t, 'slug') for t in themes]
    
    preq = 'pre-qualified' in doc_types
    n_docs = len(doc_types)
    
    return [preq, n_docs, doc_types, theme_types]
    
result_candidates_all = actions[actions.type == "result_candidates"]    
result_candidates = result_candidates_all[result_candidates_all['query'] != 'undefined']
parsed_candidates = result_candidates['result_candidates'].apply(lambda cands : read_candidates(cands))


In [ ]:
result_candidates_all.shape
result_candidates.shape

In [ ]:
parsed_candidates_df = pd.DataFrame(parsed_candidates.tolist(), columns = ['isPreq', 'nDoc', 'doc_types', 'theme_types'])

In [ ]:
parsed_candidates_df[parsed_candidates_df.theme_types.apply(lambda n : len(n)) == 0].shape

In [ ]:
parsed_candidates_df.isPreq

plot = sns.catplot(y="isPreq", kind="count", palette="ch:.25", data=parsed_candidates_df);
plt.subplots_adjust(top=0.9)
plot.fig.suptitle("Proportion de recherches prequalifiées")

In [ ]:
#actions[actions.type == "result_candidates"][['type', 'query', 'result_candidates']][:30]

In [ ]:
preq_length = parsed_candidates_df[parsed_candidates_df.isPreq].doc_types.apply(lambda x : len(x))

In [ ]:
preq_length_df = pd.DataFrame({'pre-qualified-n-results':preq_length})

In [ ]:
plot = sns.catplot(x="pre-qualified-n-results", kind="violin", data=preq_length_df, aspect=18.7/8.27);
plt.subplots_adjust(top=0.8)
plot.fig.suptitle("Nombre de résultats pour les recherches préqualifiées.")

## check number of results with less than X results

## check feedback

In [ ]:
actions[actions.type == 'feedback']
plot = sns.catplot(y="feedback_type", kind="count", palette="ch:.25", data=actions[actions.type == 'feedback'], aspect=5.7/3.27);
plt.subplots_adjust(top=0.9)
plot.fig.suptitle("Retours explicites des utilisateurs.")

In [ ]:
neg_visits = actions[actions.feedback_type == 'negative'].idVisit.unique()
actions[actions.idVisit == neg_visits[32]]

## get most frequent queries (more than 3) to add to suggestions

In [ ]:
n_unique_searches['query'].apply(lambda x: x.lower()).value_counts()[:30]

# Print ranks for most frequent suggestion selected

In [ ]:
for e in actions[actions.type=='select_suggestion'].sugg_selection.value_counts()[:50].index.tolist():
    print_rangs(e)

In [ ]:
print_rangs("rupture à l'amiable")

In [ ]:
# most frequent queries without hit
for g, rows in df_searches[df_searches['rank'] == -1].groupby(["query"]):
    n = rows.shape[0]
    if (n > 10):
        print(f'{g} -> {n}')

In [ ]:
df_searches.groupby(["query"])

In [ ]:
# test : 
# reordonner les resultats DF de maniere factice et comparer la difference entre les scores

In [ ]:
import requests

params = {
    "y_pred" : ["b", "a", "c", "w", "k","e"], # y_pred (array): list of documents id predicted by the system
    "y_true" : ["a", "b", "c", "e"], # y_true (array): documents id scored by humans sorted from most relevant to least relevant
    "y_score" : {
        "a":15,
        "b":3,
        "c":2,
        "e":2
            }, # y_score is a dict {"doc_id":"score"} of documents assigned as relevant y humans with the associated scores (ordered internally)
    "method" : "all", # one of ["precision", "recall", "dcg", "mrr", "all"]
    "k": 4 # k integer preferaly =< to length(y_pred)
}

r = requests.post("http://localhost:4545/api/score", json = params)
r.json()

# Synthèse Catherine

In [ ]:
actions.idVisit.unique().shape

In [ ]:
actions.timestamp[:2]

In [ ]:
actions[(actions.timestamp > datetime.datetime(2020, 1, 27)) & (actions.timestamp < datetime.datetime(2020, 2, 1))].idVisit.unique().shape

In [ ]:
import datetime

x = datetime.datetime(2020, 1, 27)

actions_before = actions[actions.timestamp < x]

In [ ]:
actions_before.timeSpent

In [ ]:
def run_stats(data):
    visitesUniques = data.idVisit.unique()
    print(f"Visites uniques : {visitesUniques.shape[0]}")
    visitLength = data.groupby(["idVisit", "referrerTypeName"]).size().reset_index(name='countActions')
    survol = visitLength[visitLength.countActions == 1].shape[0] / visitLength.shape[0]
    print(f"Visites survolées : {survol}")
    print(f"Actions par visite : {visitLength.countActions.mean()}")
    print(f"Téléchargements : {data[data.type == 'download'].shape[0]}")
    unique_queries = []
    for (g,r) in data[data.type == 'search'].groupby(["idVisit", "query"]):
        unique_queries.append(g[1].lower())
    print(pd.Series(unique_queries).value_counts()[0:30])

In [ ]:
run_stats(actions)

In [ ]:
actions.type.unique()

In [ ]:
visitLength = actions_before.groupby(["idVisit", "referrerTypeName"]).size().reset_index(name='countActions')

visitLength[visitLength.countActions > 1].countActions.mean()

In [ ]:
#actions_before['query'].apply(lambda x : str(x).lower()).value_counts()[:32]


In [ ]:
actions[actions.type == 'visit_content'].shape